In [1]:
# Standard Imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Utility for displaying ImageNet labels
from tensorflow.keras.applications.mobilenet_v2 import decode_predictions

# Setting matplotlib display preferences
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['axes.grid'] = False
print(f"TensorFlow version: {tf.__version__}")


TensorFlow version: 2.20.0
